# Final Model
## Project 2 - Predicting Kings County Housing Prices with Linear Regression
Flatiron Data Science Program<br>Khyatee Desai<br>October 23, 2020

### Read in Data

In [32]:
import pandas as pd
import numpy as np
from geopy import distance
from geopy import Point
from itertools import combinations
import statistics as stats
import scipy.stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import sklearn
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pickle
import warnings
warnings.filterwarnings('ignore')

In [33]:
df = pd.read_csv('kc_house_data_train.csv')

## Data Cleaning & Handling Outliers

In [34]:
### Change date strings to datetime, drop first two columns because the aren't relevant
df['yr_sold'] = pd.to_datetime(df['date'].str.slice(0,8), format='%Y%m%d', errors='ignore').dt.year
df.drop(columns=['Unnamed: 0', 'id','date'],axis=1,inplace=True)

In [35]:
### Impute Outliers
for feat in ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',  'sqft_above', 'sqft_basement','sqft_living15', 'sqft_lot15']: 
    above_6std = df[feat].mean()+(6*df[feat].std())
    # if outliers are above 6 standard devs, reduce to 6 standard devs from mean
    df[feat] = np.where(df[feat].values >above_6std, df[feat].mean()+6*df[feat].std(), df[feat])

# Feature Engineering


In [36]:
# Create dummy variables for zip code 
zip_dummies = pd.get_dummies(df['zipcode'].astype(str), dtype=int, drop_first=True)
df.drop(columns=['zipcode'],inplace=True)
new_cols = 'zip'+zip_dummies.columns
zip_dummies.columns = new_cols

In [37]:
# Create yrs_old feature
yrs_old = df['yr_sold']- df['yr_built']

In [38]:
# Distance (miles) from each house to Pikes Place Market (essentially downtown seattle)
distances=[]
for (lat, long) in list(zip(df['lat'],df["long"])):
    p1 = Point(f'{lat} {long}')
    pikes_place = Point("47.6086 -122.3401")
    distances.append(distance.distance(p1,pikes_place).miles)

In [39]:
# add new features to dataframe
new_features = pd.DataFrame()
new_features['yrs_old'] = yrs_old
new_features['miles_from_city'] = distances

### Interaction Features

In [40]:
# Generate combinations of features
y = df[['price']]
X = df.iloc[:,1:]
interactions = list(combinations(X.columns, 2))
interaction_dict = {}
for interaction in interactions:
    X_copy = X.copy()
    X_copy['interact'] = X_copy[interaction[0]] * X_copy[interaction[1]]  
    model = LinearRegression() #run model with each possible interaction
    model.fit(X_copy, y)
    interaction_dict[model.score(X_copy, y)] = interaction # add R-squared for each interaction to a dictionary

In [41]:
# Add best 60 interactions to model
top_interactions = sorted(interaction_dict.keys(), reverse = True)[:60]
for interaction in top_interactions:
    feature1 = interaction_dict[interaction][0]
    feature2 = interaction_dict[interaction][1]
    new_features[feature1+'_X_'+feature2] = X[feature1] * X[feature2] #also add to new_features df

### Log Transformations

In [42]:
# generate new features for logs of non-normal features (based on histograms)
non_normal = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'condition', 'grade', 'sqft_above', 'sqft_basement','sqft_living15', 'sqft_lot15']
for feat in non_normal:
    new_features['log_'+feat] = df[feat].map(lambda x: np.log(x))
new_features = new_features.replace([np.inf, -np.inf], 0)

### Polynomial Features¶

In [43]:
non_normal = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'condition', 'grade', 'sqft_above', 'sqft_living15', 'sqft_lot15']
for feat in non_normal:
    new_features[feat+'^2'] = df[feat]**2
    new_features[feat+'^3'] = df[feat]**3

# Feature Selection


### Recursive Feature Elimination

In [44]:
X_all = pd.concat([df.iloc[:,1:], zip_dummies, new_features], axis=1)
ols = LinearRegression()
selector = RFECV(estimator=ols, step=1, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
selector.fit(X_all, y)
rfe_features = X_all.columns[selector.support_]

In [45]:
model_final = LinearRegression()
model_final.fit(X_all[rfe_features], y)
print('r-squared: ',model_final.score(X_all[rfe_features], y))
y_pred = model_final.predict(X_all[rfe_features])
print('rmse: ',np.sqrt(mean_squared_error(y, y_pred)))

r-squared:  0.8813219725128786
rmse:  128603.46478124417


# Save final model

In [46]:
pickle_out = open("model.pickle","wb")
pickle.dump(model_final, pickle_out)
pickle_out.close()